In [7]:
!pip install opendatasets

In [9]:
import opendatasets as od

In [14]:
od.download("https://www.kaggle.com/datasets/lava18/google-play-store-apps")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: rezuwanahmed
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/lava18/google-play-store-apps


100%|██████████| 1.94M/1.94M [00:00<00:00, 454MB/s]

In [15]:
import pandas as pd
df = pd.read_csv("/content/google-play-store-apps/googleplaystore_user_reviews.csv")

In [16]:
print(df.columns)

Index(['App', 'Translated_Review', 'Sentiment', 'Sentiment_Polarity',
       'Sentiment_Subjectivity'],
      dtype='object')


In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# 1. Load the dataset (googleplaystore_user_reviews.csv)
# If on Colab, upload the file and use this path:
df = pd.read_csv('/content/google-play-store-apps/googleplaystore_user_reviews.csv')


# 2. CRITICAL CLEANING: Drop rows where Review or Sentiment is missing
df = df.dropna(subset=['Translated_Review', 'Sentiment'])

# 3. Focus on Positive and Negative only for highest accuracy
# Neutral reviews often confuse the model
df = df[df['Sentiment'] != 'Neutral']

# 4. Prepare X and y
X = df['Translated_Review'].str.lower()
y = df['Sentiment']

# 5. Vectorization with N-Grams (Bigrams)
# Looking at word pairs (like "not working") increases accuracy
tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), max_features=5000)
X_vectorized = tfidf.fit_transform(X)

# 6. Split the data
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)

# 7. Use Logistic Regression (often outperforms Naive Bayes for this specific data)
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# 8. Evaluation
y_pred = model.predict(X_test)
print(f"Final Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%\n")
print("Classification Report:\n")
print(classification_report(y_test, y_pred))

# 9. Test it!
def test_app_review(text):
    text_vec = tfidf.transform([text.lower()])
    result = model.predict(text_vec)[0]
    return result

print(f"Sample Test 1: 'This app crashes every time' -> {test_app_review('This app crashes every time')}")
print(f"Sample Test 2: 'Absolutely beautiful UI and very fast' -> {test_app_review('Absolutely beautiful UI and very fast')}")

Final Accuracy: 91.68%

Classification Report:

              precision    recall  f1-score   support

    Negative       0.92      0.75      0.82      1691
    Positive       0.92      0.98      0.95      4763

    accuracy                           0.92      6454
   macro avg       0.92      0.86      0.89      6454
weighted avg       0.92      0.92      0.91      6454

Sample Test 1: 'This app crashes every time' -> Positive
Sample Test 2: 'Absolutely beautiful UI and very fast' -> Positive
